In [ ]:
from torchvision.models import Inception_V3_Weights
weights = Inception_V3_Weights.DEFAULT
ans = torch.topk(prediction,1)[1][0][0]
print(ans)
print(weights.meta['categories'][ans])

In [2]:
from scipy.io import loadmat
from PIL import Image
import os

# Load the .mat file
data = loadmat('../dataset/car_devkit/devkit/cars_train_annos.mat')


img_dir = '../dataset/cars_train/cars_train'
filenames = [os.path.join(img_dir, fname) 
                          for fname in os.listdir(img_dir) 
                          if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]

# indices = np.random.randint(0,len(filenames),64)
# print(filenames)

X = []
Y = {}
dominant_colors = []
results = []
centres = []
counts = []
for i in range(64):
    # Open the image using PIL
    image = Image.open(filenames[i])

    # Extract the index and annotation values (adjust this to your data format)
    index = int(filenames[i].split('/')[-1].split('.')[0])
    annot = data['annotations'][0][index-1]
    x1 = int(annot[0][0])
    y1 = int(annot[1][0])
    x2 = int(annot[2][0])
    y2 = int(annot[3][0])

    # Crop the image using PIL's crop method.
    # Note: PIL's crop expects (left, upper, right, lower)
    cropped_img = image.crop((x1, y1, x2, y2))
    
    # Append the resulting NumPy array to your list
    X.append(cropped_img)

/tmp/ipykernel_2003897/360438095.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x1 = int(annot[0][0])
/tmp/ipykernel_2003897/360438095.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y1 = int(annot[1][0])
/tmp/ipykernel_2003897/360438095.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x2 = int(annot[2][0])
/tmp/ipykernel_2003897/360438095.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in fut

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import dnnlib
import legacy
from torchvision.utils import save_image
from torchvision import models

# --------------------------
# Device Setup
# --------------------------
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

# --------------------------
# Load Pretrained StyleGAN2 Network
# --------------------------
network_pkl = "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-car-config-e.pkl"
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)
G.eval()
for param in G.parameters():
    param.requires_grad = False

# --------------------------
# Load Pretrained InceptionV3 Network
# --------------------------
inception = models.inception_v3(pretrained=True).to(device)
inception.eval()
for param in inception.parameters():
    param.requires_grad = False

# --------------------------
# Define the Latent VAE for generating latent vectors
# --------------------------
class LatentVAE(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=512, latent_dim=512):
        super(LatentVAE, self).__init__()
        # Encoder layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def encode(self, x):
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

latent_vae = LatentVAE().to(device)
optimizer = optim.Adam(latent_vae.parameters(), lr=1e-4)

# --------------------------
# Preprocessing Transforms
# --------------------------
preprocess_img = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor()
])
preprocess_resize = transforms.Resize((299, 299))  # For generated images

# --------------------------
# Helper Functions
# --------------------------
def get_layer_activation(model, input_tensor, layer_name, detach=True):
    """
    Extract activations from a specified layer.
    """
    model.eval()
    activations = {}
    def hook(module, input, output):
        activations[layer_name] = output.detach() if detach else output
    target_module = None
    for name, module in model.named_modules():
        if name == layer_name:
            target_module = module
            break
    if target_module is None:
        raise ValueError(f"Layer '{layer_name}' not found in model.")
    hook_handle = target_module.register_forward_hook(hook)
    _ = model(input_tensor)
    hook_handle.remove()
    return activations[layer_name]

def gram_matrix(features):
    b, c, h, w = features.size()
    features = features.view(b, c, h * w)
    gram = torch.bmm(features, features.transpose(1, 2))
    return gram / (c * h * w)

def compute_content_loss(target, generated):
    return F.mse_loss(generated, target)

# Optionally, specify layers for style and content loss computation
selected_layers_style = ['Conv2d_1a_3x3', 'Mixed_5c', 'Mixed_6e']
selected_layers_content = ['Conv2d_1a_3x3', 'Mixed_5c', 'Mixed_6e']

# --------------------------
# Prepare the Target Image
# --------------------------
# Note: Replace this dummy image with your actual dataset image.
# dummy_image = X[15]  # Dummy image
# target_tensor = preprocess_img(dummy_image).unsqueeze(0).to(device)

# plt.imshow(target_tensor.cpu().squeeze().permute(1,2,0))
# plt.title("Target Image")
# plt.axis('off')
# plt.show()

# --------------------------
# Define Loss Function for Classification Reward
# --------------------------
# Use Binary Cross Entropy (BCE) loss where a low loss (reward) is assigned
# if InceptionV3 classifies the generated image as a sports car (label 847).
bce_loss = nn.BCELoss()



import torch.nn.functional as F  # ensure F is imported for interpolation

# --------------------------
# Training Loop with Batch Size 32
# --------------------------
batch_size = 10
num_steps = 10000
for step in range(num_steps):
    optimizer.zero_grad()
    
    # Generate a batch of random latent vectors and pass them through the VAE.
    z = torch.randn(batch_size, 512).to(device)
    latent_out, mu, logvar = latent_vae(z)
    
    # Use the decoded latent vectors with StyleGAN2's mapping network.
    w = G.mapping(latent_out, None)
    
    # Generate a batch of images using StyleGAN2's synthesis network.
    imgs = G.synthesis(w, noise_mode='const')
    imgs = (imgs.clamp(-1, 1) + 1) / 2  # Normalize images to [0, 1]
    
    # Resize generated images to 299x299 for Inception using bilinear interpolation.
    imgs_inp = F.interpolate(imgs, size=(299, 299), mode='bilinear', align_corners=False)
    
    # Forward pass through Inception to get predictions.
    predictions = inception(imgs_inp)
    pred_labels = predictions.argmax(dim=1)
    target_labels = torch.full((batch_size,), 847, dtype=torch.long).to(device)  # Target label: 847
    
    # Compute classification loss using BCE.
    # First, get binary predictions (1 if predicted label equals target label, else 0).
    # print(pred_labels)
    # break
    y_pred = (pred_labels == target_labels).float()
    classification_loss = nn.BCELoss()(y_pred, torch.ones_like(y_pred))
    
    # VAE Losses:
    # Reconstruction loss: encouraging latent_out to be close to the original sampled z.
    reconstruction_loss = F.mse_loss(latent_out, z)
    # KL divergence: regularizing the latent space to be close to a standard normal distribution.
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / batch_size
    
    # Total loss: sum of classification, reconstruction, and KL divergence losses.
    total_loss = classification_loss + reconstruction_loss + kl_divergence
    
    total_loss.backward()
    optimizer.step()
    
    # Periodically save the model and display the first generated image in the batch.
    if step % 100 == 0:
        torch.save(latent_vae.state_dict(), "latent_vae.pth")
        print(f"Step {step}: Total Loss = {total_loss.item():.4f}, "
              f"Classification Loss = {classification_loss.item():.4f}, "
              f"Reconstruction Loss = {reconstruction_loss.item():.4f}, "
              f"KL Divergence = {kl_divergence.item():.4f}")
        
        # Display the first image from the batch.
        plt.imshow(imgs[0].detach().cpu().permute(1, 2, 0))
        plt.title(f"Generated Image at Step {step}")
        plt.axis('off')
        plt.show()
